### [Google Colab] subir código cifar10.py. Basta con ejecutar una vez (inmune a reinicios de kernel de python, no de la máquina)

In [11]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))

Saving cifar10.py to cifar10 (1).py
User uploaded file "cifar10.py" with length 6703 bytes


# Aquí comienza el cuerpo del código
## Reinicie luego de entrenar cada modelo. Recuerde actualizar el directorio de tensorboard.

In [12]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import sys
import time

import numpy as np
import tensorflow as tf

from cifar10 import CIFAR10

### Constructor del dataset. Modifique aquí el *flag* de *data augmentation*

In [13]:
# Load dataset
batch_size = 64
cifar10 = CIFAR10(batch_size=batch_size, validation_proportion=0.1, augment_data=False)

## Elija aquí el directorio donde guardará los registros de Tensorboard

In [14]:
PARENT_DIR = './summaries/'
SUMMARIES_DIR = PARENT_DIR + 'mlp'

## Constructores de modelos

In [15]:
# Modelo de red convolucional
class ConvClassifier(tf.keras.Model):
    """Implementacion de clasificador en base a red neuronal convolucional.
    """
    
    def __init__(
        self,
        input_shape,
        layer_sizes,
        learning_rate=0.0005,
        dropout_rate=0.5):
        """Construye un clasificador Red Neuronal Convolucional.
        
        Args:
            input_shape: tupla que contiene dimensiones de las imagenes de entrada
                al modelo, exluyendo la dimensióón de batch. La tupla ha de tener largo 3
                y formato (Image_height, image_width, Channels)
            layer_sizes: Lista de enteros que indica la cantidad de filtros 
                convolucionales en cada capa de la red.
            learning_rate: Escalar que indica la tasa de aprendizaje.
            dropout_rate: Probabilidad con la que se apaga cada neurona de la 
                capa fully connected 1 durante el entrenamiento del modelo.
        """
        
        # Inicializa atributos propios de un objeto del tipo tf.keras.Model
        super().__init__()
        # Agregar parametros al objeto
        self.input_shape_tuple = input_shape
        self.layer_sizes = layer_sizes
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        # Inicializa capas del modelo
        self.layers_list = self._init_layers(layer_sizes)
        # Crea un objeto optimizador
        self.optimizer = tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate)
        # Crea un objeto funcion de costo-perdida
        self.loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        self.loss_function_name = 'xentropy'

    def _init_layers(self, layer_sizes):
        """Inicializa-crea los objetos de cada capa de la red CNN.

        Args:
        layer_sizes: Lista de enteros que indica la cantidad de filtros convolucionales
        de cada capa convolucional.

        Returns:
        layers_list: Lista de objetos asociados a cada capa creada.
        """   

        layers_list = []
        # Inicializacion de capas del modelo
        # Inicializacion de capas de entrada, que fija el tamaño de las entradas
        layers_list.append(tf.keras.layers.InputLayer((self.input_shape_tuple)))
        # Inicializacion de capas convolucionales

        ###########################RESIZE IMAGENES############################

        layers_list.append(
            tf.keras.layers.Conv2D(
              16, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv_primera_capa' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
              32, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv2' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
              64, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv3' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
              64, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv4' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
              64, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv5' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
              64, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv6' ))
        layers_list.append(
            tf.keras.layers.MaxPool2D())
        layers_list.append(
          tf.keras.layers.Conv2D(
              128, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv7' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
              128, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv8' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
              128, kernel_size=3, padding='same', 
              bias_initializer=tf.keras.initializers.Constant(value=0.05), 
              activation=tf.nn.relu, name='conv9' ))
        layers_list.append(
            tf.keras.layers.MaxPool2D())
        layers_list.append(
          tf.keras.layers.Conv2D(
                256, kernel_size=3, padding='same', 
                bias_initializer=tf.keras.initializers.Constant(value=0.05), 
                activation=tf.nn.relu, name='conv10' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
                256, kernel_size=3, padding='same', 
                bias_initializer=tf.keras.initializers.Constant(value=0.05), 
                activation=tf.nn.relu, name='conv11' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
                256, kernel_size=3, padding='same', 
                bias_initializer=tf.keras.initializers.Constant(value=0.05), 
                activation=tf.nn.relu, name='conv12' ))
        layers_list.append(
          tf.keras.layers.Conv2D(
                256, kernel_size=3, padding='same', 
                bias_initializer=tf.keras.initializers.Constant(value=0.05), 
                activation=tf.nn.relu, name='conv13' ))
        # Inicializacion de funcion de capa de pooling
       
        # Inicializacion de capa que aplana featuremaps de la convolucion para que 
        # puedan entrar a capas fully-connected
        layers_list.append(
            tf.keras.layers.Flatten())
        # Inicializacion de capa fully-connected 1
        layers_list.append(
            tf.keras.layers.Dense(50, activation=tf.nn.relu, name='fc1'))
        # Inicializacion de capa dropout
        layers_list.append(
            tf.keras.layers.Dropout(self.dropout_rate))
        # Inicializacion de capa fully-connected 2
        layers_list.append(
            tf.keras.layers.Dense(10, name='fc2'))
        # Inicializacion de activacion de salida
        layers_list.append(
            tf.keras.layers.Activation(tf.nn.softmax))
        return layers_list

    def call(self, x, training=False, get_logits=False):
        """Metodo que entrega la salida del modelo, 
        define el paso forward de la red neuronal, relacionando las capas 
        de la red entre si. Se define la arquitectura del modelo.

        Args:
          x: Tensor de entrada de dimensiones (batch_size, n_features).
          training: Boolean que indica si se esta en fase de entrenamiento o no.
            En este caso define el comportamiento durante entrenamiento y evaluacion de capas 
             como dropout o batch normalization 
          get_logits: Bolean que indica si como salida del modelo se obtienen los
            logits o las predicciones

        Returns:
          x: Tensor de salida de la red, de dimensiones (batch_size, n_classes).
        """
        for layer_index, layer in enumerate(self.layers_list):
            # se se desean los logits, se retorna la salida del modelo sin la ultima activacion
            if get_logits and layer_index == (len(self.layers_list)-1):
                return x 
            x = layer(x, training=training)
        return x

    @tf.function
    def train_step(self, x_data, y_labels):
        """Metodo que realiza la actualizacion por gradiente.

        Se aplica un algoritmo de optimizacion para ejecutar una
        iteracion de minimizacion por gradiente sobre el loss del modelo.

        Args:
        x_data: datos de entrenamientos sobre los que calcular la loss.
        y_labels: etiquetas de entrenamiento sobre las que calcular la loss.

        Returns:
        loss: el valor de la funcion de costo para x_data, y_labels.
        accuracy: porcentaje de aciertos entre y_labels y las predicciones sobre x_data.
        """
        with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
            logits = self.call(x_data, training=True, get_logits=True)
            loss = self.loss_object(y_labels, logits)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        return loss

    def _get_accuracy_from_logits(self, logits, labels):
        """Metodo que entrega el accuracy entre logits de prediccion y etiquetas.

        Args:
          logits: para datos de etiqueta labels.
          labels: etiquetas.

        Returns:
          accuracy: porcentaje de aciertos por los logits, respecto a los labels.
        """
        softmax_prediction = tf.nn.softmax(logits)
        correct_prediction = tf.equal(tf.argmax(softmax_prediction, 1),
                             tf.argmax(labels, 1))
        accuracy = tf.reduce_mean(
            tf.cast(correct_prediction, tf.float32),
            name='accuracy')
        return accuracy

    @tf.function
    def eval_step(self, x_data, y_labels):
        """Metodo que evalua la funcion de costo y el accuracy del modelo.

        Args:
        x_data: datos sobre los que calcular la loss y accuracy.
        y_labels: etiquetas sobre las que calcular la loss y accuracy.

        Returns:
        loss: el valor de la funcion de costo para x_data, y_labels.
        accuracy: porcentaje de aciertos entre y_labels y las predicciones sobre x_data.
        """
        logits = self.call(x_data, training=False, get_logits=True)
        loss = self.loss_object(y_labels, logits)
        accuracy = self._get_accuracy_from_logits(logits, y_labels)
        return loss, accuracy

    @tf.function
    def prediction_step(self, x_data):
        """Metodo que retorna predicciones para x_data

        Args:
          x_data: datos a predecir.

        Returns:
          predictions: predicciones para x_data
        """
        predictions = self.call(x_data, training=False, get_logits=False)
        return predictions

    def write_to_evaluation_summary(self, summary_writer, step, loss, accuracy):
        """Metodo que calcula metricas y parametros a ser monitoreados en tensorboard.
        Monitorea histograma de parametros, metricas del modelo e imagenes de filtros
        de primera capa convolucional. Para datos de evaluacion

        Args:
          summary_writer: Summary writer de evaluacion
          step: iteracion del evaluacion
          loss: valor de la funcion de costo para datos de evaluacion
          accuracy: accuracy para datos de evaluacion
        """
        with summary_writer.as_default():
            for layer_i in self.layers_list:
                layer_name = layer_i.name
                if 'fc' in layer_name or 'conv' in layer_name:
                    weights, biases = layer_i.get_weights()
                    tf.summary.histogram(layer_name + '/weights', weights, step=step)
                    tf.summary.histogram(layer_name + '/biases', biases, step=step)
                if 'conv_primera_capa' in layer_name:
                    tf.summary.image(
                        'conv1_filters',
                        tf.transpose(weights, perm=[3, 0, 1, 2]), step=step,
                        max_outputs=self.layer_sizes[0]
                    )
            tf.summary.scalar(self.loss_function_name + '_loss', loss, step=step)  
            tf.summary.scalar('accuracy', accuracy, step=step)  

    def write_to_learning_summary(self, summary_writer, step, loss, accuracy):
        """Metodo que calcula metricas a ser monitoreados en tensorboard.
        Monitorea metricas del modelo.

        Args:
          summary_writer: Summary writer de
          step: iteracion
          loss: valor de la funcion de costo para datos
          accuracy: accuracy para datos
        """
        with summary_writer.as_default():
            tf.summary.scalar(self.loss_function_name + '_loss', loss, step=step)  
            tf.summary.scalar('accuracy', accuracy, step=step)  
    

    def write_graph_to_summary(self, summary_writer, X_data, logdir):
        """Metodo que guarda el grafo de evaluacion (arquitectura del modelo)
        en tensorboard. Se guardan las operaciones involugradas en un paso forward
        o self.call del modelo

        Args:
          summary_writer: Summary writer para datos
          X_data: Datos
        """
        logdir = logdir + '/graph'
        tf.summary.trace_on(graph=True, profiler=False)
        predicted_proba = self.prediction_step(X_data)
        with summary_writer.as_default():
            tf.summary.trace_export(
                name="prediction_step",
                step=0,
                profiler_outdir=logdir)
        tf.summary.trace_off()
    
    
    def predict_proba(self, X):
        """Retorna las probabilidades de clase para los datos de entrada.
              
        Args:
            X: datos de entrada
        
        Returns:
            predicted_proba.numpy(): Arreglo numpy de las probabilidades de 
              cada clase, para cada muestra de X.
        """
        # Obtener las probabilidades de salida de cada clase
        predicted_proba = self.prediction_step(X)
        return predicted_proba.numpy()
    
    def predict_label(self, X):
        """Retorna la etiqueta predicha para los datos de entrada.
        
        Args:
            X: datos de entrada
        
        Returns:
            predicted_labels: Arreglo numpy de las predicciones de cada muestra de X.
        """
        # Obtener la probabilidad de cada clase
        predicted_proba = self.prediction_step(X)
        # Etiquetar segun la etiqueta mas probable
        predicted_labels = np.argmax(predicted_proba, axis=1)
        return predicted_labels

In [16]:

# Modelo de red MLP
class MLPClassifier(ConvClassifier):
    """Implementacion de clasificador en base a red MLP.
    """
    
    def __init__(
        self,
        learning_rate=0.0005,
        dropout_rate=0.5):
        """Construye un clasificador Perceptron Multicapa de 2 capas.
        
        Args:
            learning_rate: Escalar que indica la tasa de aprendizaje.
            dropout_rate: Probabilidad con la que se apaga cada neurona de la 
                capa fully connected 1 durante el entrenamiento del modelo.
        """
        
        # Inicializa atributos propios de un objeto del tipo tf.keras.Model
        tf.keras.Model.__init__(self)
        # Agregar parametros al objeto
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate
        # Inicializa capas del modelo
        self.layers_list = self._init_layers()
        # Crea un objeto optimizador
        self.optimizer = tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate)
        # Crea un objeto funcion de costo-perdida
        self.loss_object = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        self.loss_function_name = 'xentropy'
    
    def _init_layers(self):
        """Inicializa-crea los objetos de cada capa de la red MLP.

        Returns:
          layers_list: Lista de objetos asociados a cada capa creada.
        """
        layers_list = []
        # Inicializacion de capas del modelo
        # Inicializacion de capas de entrada, que fija el tamaño de las entradas
        layers_list.append(tf.keras.layers.Flatten())
        # Inicializacion de capas convolucionales
        layers_list.append(tf.keras.layers.Dense(100, activation=tf.nn.relu, name='fc1'))
        # Inicializacion de capa dropout
        layers_list.append(tf.keras.layers.Dropout(self.dropout_rate))
        # Inicializacion de capa fully-connected 2
        layers_list.append(tf.keras.layers.Dense(10, name='fc2'))
        # Inicializacion de activacion de salida
        layers_list.append(tf.keras.layers.Activation(tf.nn.softmax))
        return layers_list

## Inicializacion de modelos

### Aqui puede escoger que modelo usar y modificar el valor de *dropout_rate* usado al computar *train_step* para activar/desactivar el *dropout_rate*.

In [17]:
#### PARAMETROS DE MODELOS A MODIFICAR ###
use_convnet = True
n_filters_convs = [16, 32, 64, 64 ,64 ,64 ,128 ,128 ,128, 128, 128, 256,256,256,256,256]
dropout_rate = 0.5

if use_convnet:
    model = ConvClassifier(
        input_shape=(1000, 1000, 3),
        layer_sizes=n_filters_convs,
        dropout_rate=dropout_rate,
        learning_rate=0.0005)

else:
    model = MLPClassifier(
        dropout_rate=dropout_rate,
        learning_rate=0.0005)

### Construcción del optimizador + funciones auxiliares

In [18]:
# Useful training functions
def validate(model, val_summary_writer, step):
    cifar10.shuffleValidation()
    batches = cifar10.getValidationSet(asBatches=True)
    accs = []
    xent_vals = []
    for batch in batches:
        data, labels = batch
        xentropy_val, acc = model.eval_step(data, labels)
        accs.append(acc)
        xent_vals.append(xentropy_val)
    mean_xent = np.array(xent_vals).mean()    
    mean_acc = np.array(accs).mean()
    model.write_to_evaluation_summary(val_summary_writer, step, mean_xent, mean_acc)
    return mean_acc, mean_xent

def test(model):
    batches = cifar10.getTestSet(asBatches=True)
    accs = []
    for batch in batches:
        data, labels = batch
        _, acc = model.eval_step(data, labels)
        accs.append(acc)
    mean_acc = np.array(accs).mean()
    return mean_acc

# Tensorboard writers
train_writer = tf.summary.create_file_writer(SUMMARIES_DIR+'/train')
model.write_graph_to_summary(train_writer, cifar10.getValidationSet()[0], SUMMARIES_DIR)
validation_writer = tf.summary.create_file_writer(SUMMARIES_DIR+'/validation')

## Entrenar

In [ ]:
cifar10.reset()
print("Trainable variables")
for n in model.trainable_weights:
    print(n.name, ' ', n.shape)
if use_convnet:
    epochs = 50
else:
    epochs = 50
    
t_i = time.time()
n_batches = cifar10.n_batches
val_acc_vals = []
test_acc_vals = []
while cifar10.getEpoch() < epochs:
    epoch = cifar10.getEpoch()
    batch, batch_idx = cifar10.nextBatch()
    batch_data = batch[0]
    batch_labels = batch[1]
    
    # just a training iteration
    model.train_step(batch_data, batch_labels)
    
    step = batch_idx+epoch*n_batches
    
    # Write training summary
    if step % 50 == 0:
        loss, accuracy = model.eval_step(batch_data, batch_labels)
        model.write_to_learning_summary(train_writer, step, loss, accuracy)
        
    # gradient (by layer) statistics over last training batch & validation summary
    if batch_idx==0:
        loss, acc = model.eval_step(batch_data, batch_labels)
        validation_accuracy, validation_loss = validate(model, validation_writer, step)
        print('[Epoch %d, it %d] Training acc. %.3f, loss %.3f. \
Valid. acc. %.3f, loss %.3f' % (
            epoch,
            step,
            acc,
            loss,
            validation_accuracy,
            validation_loss
        ))
        val_acc_vals.append(validation_accuracy)
        test_accuracy = test(model)
        test_acc_vals.append(test_accuracy)
        print("Time elapsed %.2f minutes" % ((time.time()-t_i)/60.0))

val_acc_vals = np.array(val_acc_vals)
test_acc_vals = np.array(test_acc_vals)
best_epoch = np.argmax(val_acc_vals)
test_acc_at_best = test_acc_vals[best_epoch]
print('*'*30)
print("Testing set accuracy @ epoch %d (best validation acc): %.4f" % (best_epoch, test_acc_at_best))
print('*'*30)

Trainable variables
conv_primera_capa/kernel:0   (3, 3, 3, 16)
conv_primera_capa/bias:0   (16,)
conv2/kernel:0   (3, 3, 16, 32)
conv2/bias:0   (32,)
conv3/kernel:0   (3, 3, 32, 64)
conv3/bias:0   (64,)
conv4/kernel:0   (3, 3, 64, 64)
conv4/bias:0   (64,)
conv5/kernel:0   (3, 3, 64, 64)
conv5/bias:0   (64,)
conv6/kernel:0   (3, 3, 64, 64)
conv6/bias:0   (64,)
conv7/kernel:0   (3, 3, 64, 128)
conv7/bias:0   (128,)
conv8/kernel:0   (3, 3, 128, 128)
conv8/bias:0   (128,)
conv9/kernel:0   (3, 3, 128, 128)
conv9/bias:0   (128,)
conv10/kernel:0   (3, 3, 128, 256)
conv10/bias:0   (256,)
conv11/kernel:0   (3, 3, 256, 256)
conv11/bias:0   (256,)
conv12/kernel:0   (3, 3, 256, 256)
conv12/bias:0   (256,)
conv13/kernel:0   (3, 3, 256, 256)
conv13/bias:0   (256,)
fc1/kernel:0   (16384, 50)
fc1/bias:0   (50,)
fc2/kernel:0   (50, 10)
fc2/bias:0   (10,)
[Epoch 0, it 0] Training acc. 0.141, loss 5.166. Valid. acc. 0.095, loss 6.954
Time elapsed 0.12 minutes
[Epoch 1, it 703] Training acc. 0.047, loss 2.

## Visualizar en Tensorboard

In [ ]:
LOG_DIR = PARENT_DIR  # este es el logdir de nuestros summaries
print("Showing summaries at %s" % (LOG_DIR))

%load_ext tensorboard
%tensorboard --logdir='$LOG_DIR'

In [ ]:
%%bash
ls
ps aux | grep -e 'tensorboard'

In [ ]:
#!rm -r summaries

In [ ]:
!ls